In [1]:
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Muat file .env untuk mendapatkan URL MongoDB
load_dotenv()

mongodb_url = os.getenv('URL_LOCAL')
db_name = 'youtube_rewind_indonesia'
collection_name = 'full_labeling_all_new'

# Koneksi ke MongoDB
client = MongoClient(mongodb_url)
db = client[db_name]
collection = db[collection_name]

In [5]:
# Baca kamus dasar bahasa Indonesia
with open('../../dictionary/kamus-dasar-bahasa-indonesia-new.txt', 'r', encoding='utf-8') as f:
    kamus = set(word.strip() for word in f.readlines())

# Inisialisasi list untuk menyimpan kata yang tidak ada di kamus
no_words = []

# Fungsi untuk normalisasi teks
def normalisasi(teks):
    kata_kata = teks.split()
    after_normalisasi = []
    for kata in kata_kata:
        if kata in kamus:
            after_normalisasi.append(kata)
        else:
            no_words.append(kata)
            return None  # Jika ada kata yang tidak ditemukan di kamus, kembalikan None
    return ' '.join(after_normalisasi)

# Ambil data dari MongoDB
data = list(collection.find())

# Proses setiap dokumen
for doc in data:
    text_original = doc.get('textOriginal', '')
    after_normalisasi = normalisasi(text_original)
    collection.update_one(
        {'_id': doc['_id']},
        {'$set': {'afterNormalisasi': after_normalisasi}}
    )

# Simpan kata yang tidak ditemukan dalam file CSV
if no_words:
    df_no_words = pd.DataFrame(no_words, columns=['word'])
    df_no_words.to_csv('../../dictionary/no-word-round-3.csv', index=False)

print("Proses normalisasi selesai.")

Proses normalisasi selesai.


# Tahap Normalisasi Kembali Berdasarkan Kamus

In [4]:
# Muat data dari file no-word-clean.csv
df_clean = pd.read_csv('../../dictionary/no-word-full.csv')

# Buat kamus untuk normalisasi
normalization_dict = pd.Series(df_clean.clean.values, index=df_clean.word).to_dict()

# Fungsi untuk normalisasi teks
def normalize_text(text, normalization_dict):
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

# Koneksi ke MongoDB
client = MongoClient(mongodb_url)
db = client[db_name]
collection = db["full_labeling_all_new"]

# Cari dokumen dengan textOriginal tidak null
documents = collection.find({"textOriginal": {"$ne": None}})

# Lakukan normalisasi dan perbarui dokumen
for doc in documents:
    original_text = doc['textOriginal']
    normalized_text = normalize_text(original_text, normalization_dict)
    
    # Perbarui dokumen di MongoDB
    collection.update_one({"_id": doc["_id"]}, {"$set": {"textOriginal": normalized_text}})

print("Normalisasi selesai.")


Normalisasi selesai.
